## <span style="color:red"> **Cantidad de Clicks por dispositivo en la ventana previa a la conversion** </span>

In [ ]:
cant_clk_18_20 = rh_18_20
cant_clk_21_23 = rh_21_23

cant_clk_18_20 = cant_clk_18_20.merge( clk_18_20["ref_hash"].value_counts().to_frame().reset_index().rename( columns = {"ref_hash": "cant_clk", "index":"ref_hash"} ), how = "left", on = "ref_hash" )
cant_clk_21_23 = cant_clk_21_23.merge( clk_21_23["ref_hash"].value_counts().to_frame().reset_index().rename( columns = {"ref_hash": "cant_clk", "index":"ref_hash"} ), how = "left", on = "ref_hash" )

cant_clk_18_20.fillna( 0, inplace = True )
cant_clk_21_23.fillna( 0, inplace = True )

cant_clk_18_20.to_csv( loc_ftr + "\\cant_clk_18_20.csv", index = False )
cant_clk_21_23.to_csv( loc_ftr + "\\cant_clk_21_23.csv", index = False )

## <span style="color:red"> **Ref Type del dispositivo que realizo la instalacion** </span> 

#### <span style="color:orange"> **Mean Encoding** </span> 

In [ ]:
ref_type_18_20 = rh_18_20.copy()
ref_type_21_23 = rh_21_23.copy()

sc = ins_21_23_first_ins.loc[ ins_21_23_first_ins["ref_hash"].isin( rh_18_20["ref_hash"] ), ["ref_hash","_sc"] ]

rt_18_20_sc = ins_18_20_first_ins[ ["ref_hash","ref_type"] ].merge( sc, how = "left", on = "ref_hash" ); dl_18_20_sc.fillna( 259200, inplace = True )

X_rt = dl_18_20_sc[ ["device_language"] ]
Y_rt = dl_18_20_sc[ ["_sc"] ]

te = ce.TargetEncoder(smoothing=True)
te.fit_transform( X_rt Y_rt );

ref_type_18_20["ref_type_enc"] = te.transform( ins_18_20_first_ins[["ref_type"]].astype(str) )
ref_type_21_23["ref_type_enc"] = te.transform( ins_21_23_first_ins[["ref_type"]].astype(str) )

ref_type_18_20.to_csv( loc_ftr + "\\ref_type_18_20.csv", index = False )
ref_type_21_23.to_csv( loc_ftr + "\\ref_type_21_23.csv", index = False )

#### <span style="color:orange"> **OneHotEncoding** </span> 

In [ ]:
ref_type_18_20 = rh_18_20
ref_type_21_23 = rh_21_23

rt_ohe_18_20 = pd.get_dummies( ins_18_20_first_ins[ ["ref_type"] ].astype("category") )
rt_ohe_21_23 = pd.get_dummies( ins_21_23_first_ins[ ["ref_type"] ].astype("category") )

ref_type_18_20 = pd.concat( [ref_type_18_20, rt_ohe_18_20], axis = 1 )
ref_type_21_23 = pd.concat( [ref_type_21_23, rt_ohe_21_23], axis = 1 )

## <span style="color:red"> **DeviceLenguage del dispositivo al realizar la instlacion** </span>

#### <span style="color:orange"> **Mean Encoding** </span> (Usando el SC para el promedio)

In [ ]:
dev_lang_18_20 = rh_18_20.copy()
dev_lang_21_23 = rh_21_23.copy()

sc = ins_21_23_first_ins.loc[ ins_21_23_first_ins["ref_hash"].isin( rh_18_20["ref_hash"] ), ["ref_hash","_sc"] ]

dl_18_20_sc = ins_18_20_first_ins[ ["ref_hash","device_language"] ].merge( sc, how = "left", on = "ref_hash" ); dl_18_20_sc.fillna( 259200, inplace = True )

X_dl = dl_18_20_sc[ ["device_language"] ]
Y_dl = dl_18_20_sc[ ["_sc"] ]

te = ce.TargetEncoder(smoothing=True); te.fit( X_dl, Y_dl );

# replace nans with the global mean ( https://necromuralist.github.io/kaggle-competitions/posts/mean-encoding/ )
global_mean = Y_dl["_sc"].mean()

dev_lang_18_20["dev_language"] = te.transform( ins_18_20_first_ins[ ["device_language"] ].astype( str ) ); dev_lang_18_20["dev_language"].replace( {"nan":global_mean}, inplace = True )
dev_lang_21_23["dev_language"] = te.transform( ins_21_23_first_ins[ ["device_language"] ].astype( str ) ); dev_lang_21_23["dev_language"].replace( {"nan":global_mean}, inplace = True )

dev_lang_18_20.to_csv( loc_ftr + "\\dev_lang_18_20.csv", index = False )
dev_lang_21_23.to_csv( loc_ftr + "\\dev_lang_21_23.csv", index = False )

#### <span style="color:orange"> **OneHotEncoding** </span>

In [ ]:
dev_lang_18_20 = rh_18_20
dev_lang_21_23 = rh_21_23

#Generamos todas las categorias posibles para que tengan las mismas columnas
dl_18_20 = ins_18_20_first_ins["device_language"].drop_duplicates()
dl_21_23 = ins_21_23_first_ins["device_language"].drop_duplicates()

#Creamos la categoria. Los NAN son una categoria
dl_category = pd.api.types.CategoricalDtype( dl_18_20.append( dl_21_23 ).fillna("NoDL").drop_duplicates() ) 

dl_ohe_18_20 = pd.get_dummies( ins_18_20_first_ins[ ["device_language"] ].astype(dl_category) )
dl_ohe_21_23 = pd.get_dummies( ins_21_23_first_ins[ ["device_language"] ].astype(dl_category) )

dev_lang_18_20 = pd.concat( [dev_lang_18_20, dl_ohe_18_20], axis = 1 )
dev_lang_21_23 = pd.concat( [dev_lang_21_23, dl_ohe_21_23], axis = 1 )

dev_lang_18_20.to_csv( loc_ftr + "\\dev_lang_18_20.csv", index = False )
dev_lang_21_23.to_csv( loc_ftr + "\\dev_lang_21_23.csv", index = False )

## <span style="color:red"> **Modelo del dispositivo uilizado** </span>

#### <span style="color:orange"> **Mean Encoding** </span> (Usando el SC para el promedio) 

In [ ]:
dev_modl_18_20 = rh_18_20.copy()
dev_modl_21_23 = rh_21_23.copy()

sc = ins_21_23_first_ins.loc[ ins_21_23_first_ins["ref_hash"].isin( rh_18_20["ref_hash"] ), ["ref_hash","_sc"] ]

dm_18_20_sc = ins_18_20_first_ins[ ["ref_hash","device_model"] ].merge( sc, how = "left", on = "ref_hash" ); 

X_dm = dm_18_20_sc[ ["device_model"] ].astype(str)
Y_dm = dm_18_20_sc[ ["_sc"] ]

te = ce.TargetEncoder(smoothing=True); te.fit( X_dm, Y_dm );

# replace nans with the global mean ( https://necromuralist.github.io/kaggle-competitions/posts/mean-encoding/ )
global_mean = Y_dm["_sc"].mean()

dev_modl_18_20["device_model"] = te.transform( ins_18_20_first_ins[ ["device_model"] ].astype( str ) ); dev_modl_18_20.fillna( global_mean, inplace = True )
dev_modl_21_23["device_model"] = te.transform( ins_21_23_first_ins[ ["device_model"] ].astype( str ) ); dev_modl_21_23.fillna( global_mean, inplace = True )

dev_modl_18_20.to_csv( loc_ftr + "\\dev_modl_18_20.csv", index = False )
dev_modl_21_23.to_csv( loc_ftr + "\\dev_modl_21_23.csv", index = False )

#### <span style="color:orange"> **Mean Encoding** </span> (Usando el promedio de cuantas veces fue utilizado) 

In [ ]:
dev_modl_18_20 = rh_18_20.copy()
dev_modl_21_23 = rh_21_23.copy()

dev_modl_mean_18_20 = ins_18_20[ ["device_model","ref_hash"] ].groupby("device_model").agg("count"); dev_modl_mean_18_20["mean"] = dev_modl_mean_18_20["ref_hash"]/len(ins_18_20); del(dev_modl_mean_18_20["ref_hash"])
dev_modl_mean_21_23 = ins_21_23[ ["device_model","ref_hash"] ].groupby("device_model").agg("count"); dev_modl_mean_21_23["mean"] = dev_modl_mean_21_23["ref_hash"]/len(ins_21_23); del(dev_modl_mean_21_23["ref_hash"])

dev_modl_18_20["device_model"] = ins_18_20_first_ins["device_model"]; dev_modl_18_20 = dev_modl_18_20.merge(dev_modl_mean_18_20, how = "left", on = "device_model" )
dev_modl_21_23["device_model"] = ins_21_23_first_ins["device_model"]; dev_modl_21_23 = dev_modl_21_23.merge(dev_modl_mean_21_23, how = "left", on = "device_model" )

#Contamos a los NaNs como un "modelo"
mean_nan_18_20 = ins_18_20["device_model"].isnull().sum() / len( ins_18_20 )
mean_nan_21_23 = ins_21_23["device_model"].isnull().sum() / len( ins_21_23 )

dev_modl_18_20.fillna( mean_nan_18_20, inplace = True ); del( dev_modl_18_20["device_model"] ); dev_modl_18_20.rename( {"mean":"dev_mod"}, inplace = True )
dev_modl_21_23.fillna( mean_nan_21_23, inplace = True ); del( dev_modl_21_23["device_model"] ); dev_modl_21_23.rename( {"mean":"dev_mod"}, inplace = True )

dev_modl_18_20.to_csv( loc_ftr + "\\dev_modl_18_20.csv", index = False )
dev_modl_21_23.to_csv( loc_ftr + "\\dev_modl_21_23.csv", index = False )

## <span style="color:red"> **SessionUserAgent de los dispositivos** </span>

#### <span style="color:orange"> **Mean Encoding** </span> (Usando el SC para el promedio)

In [ ]:
ins_dsua_18_20 = rh_18_20
ins_dsua_21_23 = rh_21_23

sc = ins_21_23_first_ins.loc[ ins_21_23_first_ins["ref_hash"].isin( rh_18_20["ref_hash"] ), ["ref_hash","_sc"] ]

sua_18_20_sc = ins_18_20_first_ins[ ["ref_hash","session_user_agent"] ].merge( sc, how = "left", on = "ref_hash" ); 

X_sua = sua_18_20_sc[ ["session_user_agent"] ].astype(str)
Y_sua = sua_18_20_sc[ ["_sc"] ]

te = ce.TargetEncoder(smoothing=True); te.fit( X_sua, Y_sua );

ins_dsua_18_20["sua"] = te.transform( ins_18_20_first_ins[ ["session_user_agent"] ].astype( str ) );
ins_dsua_21_23["sua"] = te.transform( ins_21_23_first_ins[ ["session_user_agent"] ].astype( str ) ); 

ins_dsua_18_20.fillna( Y_sua["_sc"].mean(), inplace = True )
ins_dsua_21_23.fillna( Y_sua["_sc"].mean(), inplace = True )

ins_dsua_18_20.to_csv( loc_ftr + "\\ins_dsua_18_20.csv", index = False )
ins_dsua_21_23.to_csv( loc_ftr + "\\ins_dsua_21_23.csv", index = False )

# <span style="color:red"> **Source que recibio mas instalaciones por dispositivo** </span> 

#### <span style="color:orange"> **Mean Encoding** </span> (Usamos el promedio de la cantidad de veces que que es la fuente principal de algun dispositivo)

In [ ]:
srce_auc_18_20 = rh_18_20.copy()
srce_auc_21_23 = rh_21_23.copy()

main_source_18_20 = auc_18_20.groupby( ["device_id", "source_id"] ).agg( {"source_id":"count"} ).rename( columns = {"source_id":"cant_auc"} ).reset_index()
main_source_21_23 = auc_21_23.groupby( ["device_id", "source_id"] ).agg( {"source_id":"count"} ).rename( columns = {"source_id":"cant_auc"} ).reset_index()

main_source_18_20 = main_source_18_20.sort_values( by = ["device_id","cant_auc"], ascending = True ).drop_duplicates( subset = ["device_id"], keep = "last" ); del( main_source_18_20["cant_auc"] )
main_source_21_23 = main_source_21_23.sort_values( by = ["device_id","cant_auc"], ascending = True ).drop_duplicates( subset = ["device_id"], keep = "last" ); del( main_source_21_23["cant_auc"] )

srce_auc_18_20 = srce_auc_18_20.merge( main_source_18_20.rename(columns = {"device_id":"ref_hash"}), how = "left", on = "ref_hash" ); srce_auc_18_20.fillna( "no_font", inplace = True )
srce_auc_21_23 = srce_auc_21_23.merge( main_source_21_23.rename(columns = {"device_id":"ref_hash"}), how = "left", on = "ref_hash" ); srce_auc_21_23.fillna( "no_font", inplace = True )

srce_auc_18_20["to_count"] = 1; srce_auc_18_20["source_id"] = srce_auc_18_20[["source_id", "to_count"]].groupby("source_id").transform( "sum" ) / len(srce_auc_18_20); del(srce_auc_18_20["to_count"])
srce_auc_21_23["to_count"] = 1; srce_auc_21_23["source_id"] = srce_auc_21_23[["source_id", "to_count"]].groupby("source_id").transform( "sum" ) / len(srce_auc_21_23); del(srce_auc_21_23["to_count"])

srce_auc_18_20.to_csv( loc_ftr + "\\srce_auc_18_20.csv", index = False )
srce_auc_21_23.to_csv( loc_ftr + "\\srce_auc_21_23.csv", index = False )

#### <span style="color:orange"> **OneHotEncoding** </span> 

In [ ]:
srce_auc_18_20 = rh_18_20.copy()
srce_auc_21_23 = rh_21_23.copy()

main_source_18_20 = auc_18_20.groupby( ["device_id", "source_id"] ).agg( {"source_id":"count"} ).rename( columns = {"source_id":"cant_auc"} ).reset_index()
main_source_21_23 = auc_21_23.groupby( ["device_id", "source_id"] ).agg( {"source_id":"count"} ).rename( columns = {"source_id":"cant_auc"} ).reset_index()

main_source_18_20 = main_source_18_20.sort_values( by = ["device_id","cant_auc"], ascending = True ).drop_duplicates( subset = ["device_id"], keep = "last" ); del( main_source_18_20["cant_auc"] )
main_source_21_23 = main_source_21_23.sort_values( by = ["device_id","cant_auc"], ascending = True ).drop_duplicates( subset = ["device_id"], keep = "last" ); del( main_source_21_23["cant_auc"] )

srce_auc_18_20 = srce_auc_18_20.merge( main_source_18_20.rename(columns = {"device_id":"ref_hash"}), how = "left", on = "ref_hash" ); srce_auc_18_20.fillna( "no_font", inplace = True )
srce_auc_21_23 = srce_auc_21_23.merge( main_source_21_23.rename(columns = {"device_id":"ref_hash"}), how = "left", on = "ref_hash" ); srce_auc_21_23.fillna( "no_font", inplace = True )

#Ya fue controlado que quedan las mismas fuentes para ambas ventanas, no quedan columnas distintas
srce_auc_ohe_18_20 = pd.get_dummies( srce_auc_18_20[ ["source_id"] ].astype( "category" ) )
srce_auc_ohe_21_23 = pd.get_dummies( srce_auc_21_23[ ["source_id"] ].astype( "category" ) )

srce_auc_18_20 = pd.concat( [srce_auc_18_20, srce_auc_ohe_18_20], axis = 1 ); del( srce_auc_18_20["source_id"] )
srce_auc_21_23 = pd.concat( [srce_auc_21_23, srce_auc_ohe_21_23], axis = 1 ); del( srce_auc_21_23["source_id"] )

srce_auc_18_20.to_csv( loc_ftr + "\\srce_auc_18_20.csv", index = False )
srce_auc_21_23.to_csv( loc_ftr + "\\srce_auc_21_23.csv", index = False )

## <span style="color:red"> **Ref Hash** </span> 

#### <span style="color:orange"> **Mean Encoding** </span> (Usamos el promedio de la cantidad de apariciones en las instalaciones en la ventana anterior)

In [ ]:
rh_encod_18_20 = rh_18_20.copy()
rh_encod_21_23 = rh_21_23.copy()

appces_18_20_ins = ins_18_20["ref_hash"].value_counts().reset_index().rename( columns = {"index":"ref_hash", "ref_hash":"appearances"} )
appces_21_23_ins = ins_21_23["ref_hash"].value_counts().reset_index().rename( columns = {"index":"ref_hash", "ref_hash":"appearances"} )

appces_18_20_ins["ref_hash_mean"] = appces_18_20_ins["appearances"] / len( ins_18_20 ); appces_18_20_ins.drop( "appearances", axis = 1, inplace = True )
appces_21_23_ins["ref_hash_mean"] = appces_21_23_ins["appearances"] / len( ins_21_23 ); appces_21_23_ins.drop( "appearances", axis = 1, inplace = True )

rh_encod_18_20 = rh_encod_18_20.merge( appces_18_20_ins, how = "left", on = "ref_hash" )
rh_encod_21_23 = rh_encod_21_23.merge( appces_21_23_ins, how = "left", on = "ref_hash" )

rh_encod_18_20.to_csv( loc_ftr + "\\rh_encod_18_20.csv", index = False )
rh_encod_21_23.to_csv( loc_ftr + "\\rh_encod_21_23.csv", index = False )

## <span style="color:red"> **Hora mas popular del usuario en las instalaciones** </span> 

Le aplicamos la transformacion sin( (hora * pi)/24 ) para agregarle periodicidad. 

In [ ]:
main_ihr_18_20 = rh_18_20.copy()
main_ihr_21_23 = rh_21_23.copy()

hour_mode_18_20 = ins_18_20[["ref_hash"]]; hour_mode_18_20["hour"] = pd.to_datetime( ins_18_20["date"] ).dt.hour
hour_mode_21_23 = ins_21_23[["ref_hash"]]; hour_mode_21_23["hour"] = pd.to_datetime( ins_21_23["date"] ).dt.hour

hour_mode_18_20 = hour_mode_18_20.groupby( ["ref_hash", "hour"] ).agg( { "hour":"count" } ).rename( columns = {"hour":"count"} ).reset_index()
hour_mode_21_23 = hour_mode_21_23.groupby( ["ref_hash", "hour"] ).agg( { "hour":"count" } ).rename( columns = {"hour":"count"} ).reset_index()

hour_mode_18_20 = hour_mode_18_20.sort_values( ["ref_hash","count"], ascending = False ).drop_duplicates( subset = ["ref_hash"], keep = "first" ).drop("count",axis = 1)
hour_mode_21_23 = hour_mode_21_23.sort_values( ["ref_hash","count"], ascending = False ).drop_duplicates( subset = ["ref_hash"], keep = "first" ).drop("count",axis = 1)

main_ihr_18_20 = main_ihr_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" ).rename( columns = {"hour":"main_hour"} )
main_ihr_21_23 = main_ihr_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" ).rename( columns = {"hour":"main_hour"} )

main_ihr_18_20["main_hour"] =  main_ihr_18_20["main_hour"].apply( lambda x: np.sin( (x*np.pi)/24 ) )
main_ihr_21_23["main_hour"] =  main_ihr_21_23["main_hour"].apply( lambda x: np.sin( (x*np.pi)/24 ) )

main_ihr_18_20.to_csv( loc_ftr + "\\main_ihr_18_20.csv", index = False )
main_ihr_21_23.to_csv( loc_ftr + "\\main_ihr_21_23.csv", index = False )

## <span style="color:red"> **Hora mas popular del usuario en las subastas** </span> 

Le aplicamos la transformacion sin( (hora * pi)/24 ) para agregarle periodicidad. 

In [ ]:
main_ahr_18_20 = rh_18_20.copy()
main_ahr_21_23 = rh_21_23.copy()

hour_mode_18_20 = auc_18_20[["device_id"]]; hour_mode_18_20["hour"] = pd.to_datetime( auc_18_20["date"] ).dt.hour
hour_mode_21_23 = auc_21_23[["device_id"]]; hour_mode_21_23["hour"] = pd.to_datetime( auc_21_23["date"] ).dt.hour

hour_mode_18_20 = hour_mode_18_20.groupby( ["device_id", "hour"] ).agg( { "hour":"count" } ).rename( columns = {"hour":"count"} ).reset_index()
hour_mode_21_23 = hour_mode_21_23.groupby( ["device_id", "hour"] ).agg( { "hour":"count" } ).rename( columns = {"hour":"count"} ).reset_index()

hour_mode_18_20 = hour_mode_18_20.sort_values( ["device_id","count"], ascending = False ).drop_duplicates( subset = ["device_id"], keep = "first" ).rename( columns = {"device_id":"ref_hash"} ).drop("count",axis = 1)
hour_mode_21_23 = hour_mode_21_23.sort_values( ["device_id","count"], ascending = False ).drop_duplicates( subset = ["device_id"], keep = "first" ).rename( columns = {"device_id":"ref_hash"} ).drop("count",axis = 1)

main_ahr_18_20 = main_ahr_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" ).rename( columns = {"hour":"main_hour"} )
main_ahr_21_23 = main_ahr_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" ).rename( columns = {"hour":"main_hour"} )

main_ahr_18_20["main_hour"] =  main_ahr_18_20["main_hour"].apply( lambda x: np.sin( (x*np.pi)/24 ) )
main_ahr_21_23["main_hour"] =  main_ahr_21_23["main_hour"].apply( lambda x: np.sin( (x*np.pi)/24 ) )

main_ahr_18_20.to_csv( loc_ftr + "\\main_ahr_18_20.csv", index = False )
main_ahr_21_23.to_csv( loc_ftr + "\\main_ahr_21_23.csv", index = False )

Son muchos NaNs, por lo que no se pueden completar con nada coherente. Esto agrega demasiado ruido al set de entrenamiento

## <span style="color:green"> **Tipo de la primera instalacion (Atribuida/No Atribuida)** </span>

#### <span style="color:orange"> **OneHotEncoding** </span> - <span style="color:red"> **No Mejora este tipo de codificacion** </span>

In [ ]:
atbt_ins_18_20 = rh_18_20.copy()
atbt_ins_21_23 = rh_21_23.copy()

atbt_ins_ohe_18_20 = pd.get_dummies( ins_18_20_first_ins[ ["attributed"] ].astype( str ) )
atbt_ins_ohe_21_23 = pd.get_dummies( ins_21_23_first_ins[ ["attributed"] ].astype( str ) )

atbt_ins_ohe_18_20 = pd.concat( [atbt_ins_18_20, atbt_ins_ohe_18_20], axis = 1 )
atbt_ins_ohe_21_23 = pd.concat( [atbt_ins_21_23, atbt_ins_ohe_21_23], axis = 1 )

atbt_ins_18_20.to_csv( loc_ftr + "\\atbt_ins_18_20.csv", index = False )
atbt_ins_21_23.to_csv( loc_ftr + "\\atbt_ins_21_23.csv", index = False )

#### <span style="color:orange"> **Mean Encoding** </span> (Usando como promedio la cantidad de veces que la primera instalacion fue de ese determinado tipo sobre el total de instalaciones)

In [ ]:
atbt_ins_18_20 = rh_18_20.copy()
atbt_ins_21_23 = rh_21_23.copy()

atbt_ins_18_20["atbt"] = ins_18_20_first_ins["attributed"]; atbt_ins_18_20["cant_ins"] = atbt_ins_18_20.groupby( "atbt" ).transform( "count" ); atbt_ins_18_20["atbt_mean"] = atbt_ins_18_20["cant_ins"]/len(rh_18_20)
atbt_ins_21_23["atbt"] = ins_21_23_first_ins["attributed"]; atbt_ins_21_23["cant_ins"] = atbt_ins_21_23.groupby( "atbt" ).transform( "count" ); atbt_ins_21_23["atbt_mean"] = atbt_ins_21_23["cant_ins"]/len(rh_21_23)

atbt_ins_18_20.drop( ["atbt","cant_ins"], axis = 1, inplace = True )
atbt_ins_21_23.drop( ["atbt","cant_ins"], axis = 1, inplace = True )

atbt_ins_18_20.to_csv( loc_ftr + "\\atbt_ins_18_20.csv", index = False )
atbt_ins_21_23.to_csv( loc_ftr + "\\atbt_ins_21_23.csv", index = False )